In [7]:
import yaml

class Config:
    def __init__(self, config_file):
        self.config_file = config_file
        self.config = self.load_config()
        print("Loaded configuration:", self.config)

    def load_config(self):
        with open(self.config_file, 'r') as f:
            config = yaml.safe_load(f)
        return config

    def get_database_config(self):
        try:
            db_type = self.config['database']['type']
            print("Database type:", db_type)
        except KeyError:
            raise KeyError("No database type specified in configuration")
        
        try:
            db_config = self.config['database'][db_type.lower()]
            print("Original Database config:", db_config)
            db_config_with_type = db_config.copy()  # Create a copy to avoid modifying the original
            db_config_with_type['type'] = db_type  # Add the type to the config
            print("Modified Database config with type:", db_config_with_type)
            return db_config_with_type
        except KeyError:
            raise KeyError(f"No configuration found for database type '{db_type}'")

    def get_param(self, query_name):
        try:
            query = self.config['queries'][query_name]
            print("Query found:", query_name, "->", query)
            return query
        except KeyError:
            raise KeyError(f"No query found with name '{query_name}'")

# Test the get_database_config function
if __name__ == "__main__":
    config = Config("configs/config.yaml")
    db_config = config.get_database_config()
    print("Final Database config:", db_config)
    query = config.get_param('LOCAL_TO_DBBRUT_QUERY')
    print("Query:", query)


Loaded configuration: {'Audit_JDBC': {'AUDIT_JDBC_connection_userPassword_password': 'Sa*201299', 'AUDIT_JDBC_drivers': 'C:/Users/sonia/Downloads/mysql-connector-j-9.0.0/mysql-connector-j-9.0.0/mysql-connector-j-9.0.0.jar', 'AUDIT_JDBC_connection_driverClass': 'com.mysql.cj.jdbc.Driver', 'AUDIT_JDBC_mappingFile': 'mysql_id', 'AUDIT_JDBC_connection_userPassword_userId': 'root', 'AUDIT_JDBC_connection_jdbcUrl': 'jdbc:mysql://localhost:3306/sqops_dataraise?allowLoadLocalInfile=true'}, 'queries': {'LOCAL_TO_DBBRUT_QUERY': 'select distinct PROJECT_NAME, JOB_NAME, JOB_PATH, JOB_VERSION, Talend_Version from audit_jobs_delta where talend_version is not null and niveau is not null', 'LOCAL_TO_DBBRUT_FILTER_name_elementNode': "where aud_nameElementNode in ('DBNAME','TYPE','QUERY','TABLE','FILENAME','TEMPDIR','sql','query')", 'LOCAL_TO_DBBRUT_FILTER_columnName_metadata': "where aud_columnName not in ('errorCode','errorMessage')", 'LOCAL_TO_DBBRUT_QUERY_ROUTINES': 'select distinct PROJECT_NAME, RO

In [8]:
import pymysql
import psycopg2
import sqlite3

class Database:
    def __init__(self, db_config):
        self.db_config = db_config
        self.connection = None
        self.cursor = None

    def connect(self):
        db_type = self.db_config['type']
        if db_type == 'mysql':
            self.connection = pymysql.connect(
                host=self.db_config['host'],
                port=self.db_config['port'],
                user=self.db_config['user'],
                password=self.db_config['password'],
                database=self.db_config['dbname']
            )
        elif db_type == 'postgresql':
            self.connection = psycopg2.connect(
                host=self.db_config['host'],
                port=self.db_config['port'],
                user=self.db_config['user'],
                password=self.db_config['password'],
                database=self.db_config['dbname']
            )
        elif db_type == 'sqlite':
            self.connection = sqlite3.connect(self.db_config['filepath'])
        else:
            raise ValueError(f"Unsupported database type: {db_type}")

        self.cursor = self.connection.cursor()

    def execute_query(self, query):
        if not self.connection:
            raise ValueError("Database connection is not established. Call connect() method first.")
        
        self.cursor.execute(query)
        return self.cursor.fetchall()

    def close(self):
        if self.cursor:
            self.cursor.close()
        if self.connection:
            self.connection.close()


In [9]:
from config import Config
from database import Database

# Example usage
if __name__ == "__main__":
    # Load configuration
    config = Config("configs/config.yaml")

    # Get selected database configuration and connect
    db_config = config.get_database_config()
    db = Database(db_config)
    db.connect()

    # Execute a query
    query = config.get_param('LOCAL_TO_DBBRUT_QUERY')
    results = db.execute_query(query)
    print("Query Results:", results)

    # Close the database connection
    db.close()


Loaded configuration: {'Audit_JDBC': {'AUDIT_JDBC_connection_userPassword_password': 'Sa*201299', 'AUDIT_JDBC_drivers': 'C:/Users/sonia/Downloads/mysql-connector-j-9.0.0/mysql-connector-j-9.0.0/mysql-connector-j-9.0.0.jar', 'AUDIT_JDBC_connection_driverClass': 'com.mysql.cj.jdbc.Driver', 'AUDIT_JDBC_mappingFile': 'mysql_id', 'AUDIT_JDBC_connection_userPassword_userId': 'root', 'AUDIT_JDBC_connection_jdbcUrl': 'jdbc:mysql://localhost:3306/sqops_dataraise?allowLoadLocalInfile=true'}, 'queries': {'LOCAL_TO_DBBRUT_QUERY': 'select distinct PROJECT_NAME, JOB_NAME, JOB_PATH, JOB_VERSION, Talend_Version from audit_jobs_delta where talend_version is not null and niveau is not null', 'LOCAL_TO_DBBRUT_FILTER_name_elementNode': "where aud_nameElementNode in ('DBNAME','TYPE','QUERY','TABLE','FILENAME','TEMPDIR','sql','query')", 'LOCAL_TO_DBBRUT_FILTER_columnName_metadata': "where aud_columnName not in ('errorCode','errorMessage')", 'LOCAL_TO_DBBRUT_QUERY_ROUTINES': 'select distinct PROJECT_NAME, RO

AttributeError: 'Database' object has no attribute 'connect'

In [ ]:
#C:/Users/sonia/Desktop/SQOPS_SERVER_PY/AUD_404_AGG_TAGGREGATE.py
from config import Config
from database import Database

def main():
    # Initialize the Config class with your configuration file
    config_file = "configs/config.yaml"
    config = Config(config_file)

    try:
        # Get MySQL database configuration and connect
        db_config = config.get_database_config()
        db = Database(db_config)

        # Connect to the database
        db.connect()

        # Test get_execution_date
        execution_date_query = config.get_param('queries', 'TRANSVERSE_QUERY_LASTEXECUTIONDATE')
        execution_date = db.get_execution_date(execution_date_query)
        print("Execution Date:", execution_date)

        # Test insert_data using table name 'aud_agg_aggregate'
        table_name = 'aud_agg_aggregate'
        insert_query = config.get_param("insert_queries",table_name)
        db.insert_data(insert_query, table_name)

    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        # Close the database connection in the finally block to ensure it's always closed
        if db:
            db.close()

if __name__ == "__main__":
    main()


In [ ]:
from config import Config  # Assuming Config class is defined in config.py
from database import Database  # Assuming Database class is defined in database.py

def main():
    config_file = "configs/config.yaml"
    config = Config(config_file)

    jobs_paths = []  # Initialize the list to store job item paths

    try:
        # Retrieve JDBC parameters and create a Database instance
        jdbc_params = config.get_jdbc_parameters()
        print("JDBC Parameters:", jdbc_params)  # Print JDBC parameters to debug

        db = Database(jdbc_params)
        db.set_jdbc_parameters(jdbc_params)  # Set JDBC parameters if needed
        db.connect_JDBC()  # Test the JDBC connection

        # Step 1: Get the execution date
        execution_date_query = config.get_param('queries', 'TRANSVERSE_QUERY_LASTEXECUTIONDATE')
        print("Execution Date Query:", execution_date_query)  # Debug print
        execution_date = db.get_execution_date(execution_date_query)
        print("Execution Date:", execution_date)

        # Step 2: Execute LOCAL_TO_DBBRUT_QUERY
        local_to_dbbrut_query = config.get_param('queries', 'LOCAL_TO_DBBRUT_QUERY')
        print("Executing query:", local_to_dbbrut_query)  # Debug print
        local_to_dbbrut_query_results = db.execute_query(local_to_dbbrut_query)
        print("local_to_dbbrut_query_results", local_to_dbbrut_query_results)

        # Step 3: Delete the output from aud_elementvaluenode based on the query results
        for result in local_to_dbbrut_query_results:
            project_name, job_name, JOB_PATH, JOB_VERSION, Talend_Version = result  # Assuming result contains these fields in order
            print(f"Deleting records for PROJECT_NAME: {project_name}, JOB_NAME: {job_name}")  # Debug print
            db.delete_records(project_name, job_name)

            # Build the job item path
            job_item_path = f"{JOB_PATH}{job_name}{JOB_VERSION}.item"
            jobs_paths.append(job_item_path)

        # Step 4: Execute NOT_AUDITED_JOBS_QUERY
        not_audited_jobs_query = config.get_param('queries', 'NOT_AUDITED_JOBS_QUERY')
        print("Executing query:", not_audited_jobs_query)  # Debug print
        not_audited_jobs_query_results = db.execute_query(not_audited_jobs_query)
        print("not_audited_jobs_query_results", not_audited_jobs_query_results)

        # Step 5: Delete the output from aud_elementvaluenode based on the query results
        for result in not_audited_jobs_query_results:
            project_name, job_name = result
            print(f"Deleting records for PROJECT_NAME: {project_name}, JOB_NAME: {job_name}")  # Debug print
            db.delete_records(project_name, job_name)

        # Step 6: Output job item paths
        for path in jobs_paths:
            print(f"Job Item Path: {path}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

    finally:
        if db:
            db.close()  # Ensure the database connection is closed

if __name__ == "__main__":
    main()


In [ ]:
import xml.etree.ElementTree as ET

# Load and parse the XML file
tree = ET.parse('C:/Users/sonia/Desktop/sqops/data/tFileInputXML_tFileOutputXML_0.1.txt')
root = tree.getroot()

# Extract and print data from `node` elements
for node in root.iter():
    if node.tag == 'node':
        # Extract and print the attributes of the `node`
        comp_data = {
            'componentName': node.get('componentName'),
            'componentVersion': node.get('componentVersion'),
            'offsetLabelX': node.get('offsetLabelX'),
            'offsetLabelY': node.get('offsetLabelY'),
            'posX': node.get('posX'),
            'posY': node.get('posY'),
        }
        print("Node Data:", comp_data)

        # Extract and print `elementParameter` data
        for elem_param in node.findall('.//elementParameter'):
            elem_data = {
                'field': elem_param.get('field'),
                'name': elem_param.get('name'),
                'show': elem_param.get('show'),
                'value': elem_param.get('value')
            }
            print("Element Parameter Data:", elem_data)

            # Extract and print `elementValue` data within `elementParameter`
            for elem_value in elem_param.findall('.//elementValue'):
                value_data = {
                    'elementRef': elem_value.get('elementRef'),
                    'value': elem_value.get('value')
                }
                print("Element Value Data:", value_data)


In [ ]:
import xml.etree.ElementTree as ET

class XMLParser:
    def __init__(self, file_path):
        """Initialize the XMLParser with the path to the XML file."""
        self.file_path = file_path
        self.tree = ET.parse(file_path)
        self.root = self.tree.getroot()

    def parse_nodes(self):
        """Parse and print data from `node` elements."""
        for node in self.root.iter():
            if node.tag == 'node':
                # Extract and print the attributes of the `node`
                data = {
                    'componentName': node.get('componentName'),
                    'componentVersion': node.get('componentVersion'),
                    'offsetLabelX': node.get('offsetLabelX'),
                    'offsetLabelY': node.get('offsetLabelY'),
                    'posX': node.get('posX'),
                    'posY': node.get('posY'),
                }
                print("Node Data:", comp_data)

                # Extract and print `elementParameter` data
                for elem_param in node.findall('.//elementParameter'):
                    elem_data = {
                        'field': elem_param.get('field'),
                        'name': elem_param.get('name'),
                        'show': elem_param.get('show'),
                        'value': elem_param.get('value')
                    }
                    print("Element Parameter Data:", elem_data)

                    # Extract and print `elementValue` data within `elementParameter`
                    for elem_value in elem_param.findall('.//elementValue'):
                        value_data = {
                            'elementRef': elem_value.get('elementRef'),
                            'value': elem_value.get('value')
                        }
                        print("Element Value Data:", value_data)

# Example usage
if __name__ == "__main__":
    # Initialize the parser with the path to the XML file
    parser = XMLParser('C:/Users/sonia/Desktop/sqops/data/tFileInputXML_tFileOutputXML_0.1.txt')
    
    # Call the method to parse and print node data
    parser.parse_nodes()


In [ ]:
import os
from config import Config  # Assuming Config class is defined in config.py
from database import Database  # Assuming Database class is defined in database.py
from XML_parse import XMLParser  # Importing the XMLParser class

def main():
    config_file = "configs/config.yaml"
    config = Config(config_file)
    items_directory = "C:/Users/sonia/Desktop/sqops/process"

    try:
        # Retrieve JDBC parameters and create a Database instance
        jdbc_params = config.get_jdbc_parameters()
        print("JDBC Parameters:", jdbc_params)  # Print JDBC parameters to debug

        db = Database(jdbc_params)
        db.set_jdbc_parameters(jdbc_params)  # Set JDBC parameters if needed
        db.connect_JDBC()  # Test the JDBC connection

        # Step 1: Get the execution date
        execution_date_query = config.get_param('queries', 'TRANSVERSE_QUERY_LASTEXECUTIONDATE')
        execution_date = db.get_execution_date(execution_date_query)
        print("Execution Date:", execution_date)

        # Step 2: Execute LOCAL_TO_DBBRUT_QUERY
        local_to_dbbrut_query = config.get_param('queries', 'LOCAL_TO_DBBRUT_QUERY')
        print("Executing query:", local_to_dbbrut_query)  # Print the query before execution
        local_to_dbbrut_query_results = db.execute_query(local_to_dbbrut_query)
        print("local_to_dbbrut_query_results", local_to_dbbrut_query_results)

        # Step 3: Delete the output from aud_elementvaluenode based on the query results
        for result in local_to_dbbrut_query_results:
            project_name, job_name, _, _, _ = result  # Assuming result contains these fields in order
            db.delete_records(project_name, job_name)
            print(f"Deleted records for PROJECT_NAME: {project_name}, JOB_NAME: {job_name}")

        # Step 4: Execute NOT_AUDITED_JOBS_QUERY
        not_audited_jobs_query = config.get_param('queries', 'NOT_AUDITED_JOBS_QUERY')
        print("Executing query:", not_audited_jobs_query)  # Print the query before execution
        not_audited_jobs_query_results = db.execute_query(not_audited_jobs_query)
        print("not_audited_jobs_query_results", not_audited_jobs_query_results)

        # Step 5: Delete the output from aud_elementvaluenode based on the query results
        for result in not_audited_jobs_query_results:
            project_name, job_name = result
            db.delete_records(project_name, job_name)
            print(f"Deleted records for PROJECT_NAME: {project_name}, JOB_NAME: {job_name}")

  # Step 6: Loop over `.item` files in the items_directory and parse them
        for filename in os.listdir(items_directory):
            if filename.endswith('.item'):
                file_path = os.path.join(items_directory, filename)
                parts = filename.split('.', 1)  # Split at the first dot
                project_name = parts[0]
                job_name = parts[1].replace('.item', '') if len(parts) > 1 else None
                print("parts:", parts)
                print("project_name:", project_name)
                print("job_name:", job_name)
                print(f"Parsing file: {file_path}")
                xml_parser = XMLParser(file_path)
                xml_parser.parse_nodes()  # Assuming this method will be enhanced to store data for further processing
    except Exception as e:
        print(f"An error occurred: {str(e)}")

    finally:
        if db:
            db.close()  # Ensure the database connection is closed

if __name__ == "__main__":
    main()


In [ ]:
import os
from config import Config  # Assuming Config class is defined in config.py
from database import Database  # Assuming Database class is defined in database.py
from XML_parse import XMLParser  # Importing the XMLParser class

def main():
    config_file = "configs/config.yaml"
    config = Config(config_file)
    items_directory = config.get_param('Directories', 'items_directory')

    db = None

    try:
        # Retrieve JDBC parameters and create a Database instance
        jdbc_params = config.get_jdbc_parameters()
        print("JDBC Parameters:", jdbc_params)  # Print JDBC parameters to debug

        db = Database(jdbc_params)
        db.set_jdbc_parameters(jdbc_params)  # Set JDBC parameters if needed
        db.connect_JDBC()  # Test the JDBC connection

        # Step 1: Get the execution date
        execution_date_query = config.get_param('queries', 'TRANSVERSE_QUERY_LASTEXECUTIONDATE')
        execution_date = db.get_execution_date(execution_date_query)
        print("Execution Date:", execution_date)

        # Step 2: Execute LOCAL_TO_DBBRUT_QUERY
        local_to_dbbrut_query = config.get_param('queries', 'LOCAL_TO_DBBRUT_QUERY')
        print("Executing query:", local_to_dbbrut_query)  # Print the query before execution
        local_to_dbbrut_query_results = db.execute_query(local_to_dbbrut_query)
        print("local_to_dbbrut_query_results:", local_to_dbbrut_query_results)

        # Step 3: Delete the output from aud_elementvaluenode based on the query results
        for result in local_to_dbbrut_query_results:
            project_name, job_name, _, _, _ = result  # Assuming result contains these fields in order
            conditions = {
                'NameProject': project_name,
                'NameJob': job_name
            }
            db.delete_records('aud_elementvaluenode', conditions)
            print(f"Deleted records for PROJECT_NAME: {project_name}, JOB_NAME: {job_name}")

        # Step 4: Execute aud_elementnode query
        aud_elementnode = config.get_param('queries', 'aud_elementnode')
        print("Executing query:", aud_elementnode)  # Print the query before execution
        aud_elementnode_results = db.execute_query(aud_elementnode)
        print("aud_elementnode_results:", aud_elementnode_results)

        # Step 5: Delete the output from aud_elementvaluenode based on the query results
        for result in aud_elementnode_results:
            project_name, job_name = result
            
            # Prepare the conditions dictionary
            conditions = {
                'NameProject': project_name,
                'NameJob': job_name
            }
            
            db.delete_records('aud_elementvaluenode', conditions)
            print(f"Deleted records for PROJECT_NAME: {project_name}, JOB_NAME: {job_name}")

        # Step 6: Loop over `.item` files in the items_directory and parse them
        for filename in os.listdir(items_directory):
            if filename.endswith('.item'):
                file_path = os.path.join(items_directory, filename)
                parts = filename.split('.', 1)  # Split at the first dot
                project_name = parts[0]
                job_name = parts[1].replace('.item', '') if len(parts) > 1 else None
                print("parts:", parts)
                print("project_name:", project_name)
                print("job_name:", job_name)
                print(f"Parsing file: {file_path}")
                xml_parser = XMLParser(file_path)
                parsed_data = xml_parser.parse_nodes()
                print('parsed data:', parsed_data)
                
                # Step 7: Insert parsed data into the `aud_element_node` table
                for data in parsed_data:
                    componentName = data['componentName']
                    field = data['field']
                    name = data['name']
                    show = data['show']
                    value = data['value']
                    
                    # Adjust the value of `Componement_UniqueName` as needed, if it's derived from `componentName`
                    Componement_UniqueName = value if field == 'TEXT' and name == 'UNIQUE_NAME' else Componement_UniqueName
                    insert_query = config.get_param('insert_queries', 'aud_elementnode')
                    params = (componentName, field, name, show, value, Componement_UniqueName, project_name, job_name, execution_date)
                    db.insert_data(insert_query, 'aud_elementnode', params)
                    print(f"Inserted data for component: {data['componentName']} into aud_elementnode")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        if db:
            db.close()  # Ensure the database connection is closed

if __name__ == "__main__":
    main()


In [ ]:
import xml.etree.ElementTree as ET

class XMLParser:
    def __init__(self, file_path):
        """Initialize the XMLParser with the path to the XML file."""
        self.file_path = file_path
        self.tree = ET.parse(file_path)
        self.root = self.tree.getroot()

    def parse_nodes(self):
        """Parse and return data from `node` elements."""
        parsed_data = []

        for node in self.root.iter():
            if node.tag == 'node':
                # Extract attributes of the `node`
                comp_data = {
                    'componentName': node.get('componentName'),
                    'componentVersion': node.get('componentVersion'),
                    'offsetLabelX': node.get('offsetLabelX'),
                    'offsetLabelY': node.get('offsetLabelY'),
                    'posX': node.get('posX'),
                    'posY': node.get('posY'),
                }

                # Extract `elementParameter` data
                for elem_param in node.findall('.//elementParameter'):
                    elem_data = {
                        'componentName': comp_data['componentName'],
                        'field': elem_param.get('field'),
                        'name': elem_param.get('name'),
                        'show': elem_param.get('show'),
                        'value': elem_param.get('value'),
                    }

                    parsed_data.append(elem_data)
                    
        return parsed_data

    def parse_context(self):
        """Parse and return data from `context` elements."""
        context_data = []

        for context in self.root.iter('context'):
            context_entry = {
                'confirmationNeeded': context.get('confirmationNeeded'),
                'name': context.get('name'),
                'contextParameters': []
            }

            # Loop through context parameters
            for context_param in context.findall('contextParameter'):
                param_data = {
                    'comment': context_param.get('comment'),
                    'name': context_param.get('name'),
                    'prompt': context_param.get('prompt'),
                    'promptNeeded': context_param.get('promptNeeded'),
                    'type': context_param.get('type'),
                    'value': context_param.get('value'),
                    'repositoryContextId': context_param.get('repositoryContextId')
                }
                context_entry['contextParameters'].append(param_data)

            context_data.append(context_entry)
        
        return context_data

# Example usage
if __name__ == "__main__":
    xml_parser =  XMLParser("C:/Users/sonia/Desktop/TOS_ESB/Studio/workspace/SERVER/process/ANALYSE/LOCAL_TO_BRUT/AUD_300_ALIMALL_BYJOB_0.1.item")
    nodes_data = xml_parser.parse_nodes()
    context_data = xml_parser.parse_context()
    print("Nodes Data:", nodes_data)
    print("Context Data:", context_data)


In [ ]:
import os
from config import Config  # Assuming Config class is defined in config.py
from database import Database  # Assuming Database class is defined in database.py
from XML_parse import XMLParser  # Importing the XMLParser class

def main():
    config_file = "configs/config.yaml"
    config = Config(config_file)
    items_directory = config.get_param('Directories', 'items_directory')

    db = None

    try:
        # Retrieve JDBC parameters and create a Database instance
        jdbc_params = config.get_jdbc_parameters()
        print("JDBC Parameters:", jdbc_params)  # Print JDBC parameters to debug

        db = Database(jdbc_params)
        db.set_jdbc_parameters(jdbc_params)  # Set JDBC parameters if needed
        db.connect_JDBC()  # Test the JDBC connection

        # Step 1: Get the execution date
        execution_date_query = config.get_param('queries', 'TRANSVERSE_QUERY_LASTEXECUTIONDATE')
        execution_date = db.get_execution_date(execution_date_query)
        print("Execution Date:", execution_date)

        # Step 2: Execute LOCAL_TO_DBBRUT_QUERY
        local_to_dbbrut_query = config.get_param('queries', 'LOCAL_TO_DBBRUT_QUERY')
        print("Executing query:", local_to_dbbrut_query)  # Print the query before execution
        local_to_dbbrut_query_results = db.execute_query(local_to_dbbrut_query)
        print("local_to_dbbrut_query_results:", local_to_dbbrut_query_results)

        # Step 3: Delete the output from aud_contextjob based on the query results
        for result in local_to_dbbrut_query_results:
            project_name, job_name, _, _, _ = result  # Assuming result contains these fields in order
            table='aud_contextjob'
            db.delete_records(table,project_name, job_name)
            print(f"Deleted records for PROJECT_NAME from  {table} : {project_name}, JOB_NAME: {job_name}")

        # Step 4: Execute aud_contextjob query
        aud_contextjob = config.get_param('queries', 'aud_contextjob')
        print("Executing query:", aud_contextjob)  # Print the query before execution
        aud_contextjob_results = db.execute_query(aud_contextjob)
        print("aud_elementnode_results:", aud_contextjob_results)

        # Step 5: Delete the output from aud_contextjob based on the query results
        for result in aud_contextjob_results:
            project_name, job_name = result
            db.delete_records(table,project_name, job_name)
            print(f"Deleted records for PROJECT_NAME from {table}: {project_name}, JOB_NAME: {job_name}")

        # Step 6: Loop over `.item` files in the items_directory and parse them
        for filename in os.listdir(items_directory):
            if filename.endswith('.item'):
                file_path = os.path.join(items_directory, filename)
                parts = filename.split('.', 1)  # Split at the first dot
                project_name = parts[0]
                job_name = parts[1].replace('.item', '') if len(parts) > 1 else None
                print("parts:", parts)
                print("project_name:", project_name)
                print("job_name:", job_name)
                print(f"Parsing file: {file_path}")
                xml_parser = XMLParser(file_path)
                parsed_data = xml_parser.parse_context()
                print('parsed context:', parsed_data)
                
                # Step 7: Insert parsed data into the `aud_contextjob` table
                for context in parsed_data:
                    environementContextName = context['name']
                    for param in context['contextParameters']:
                        comment = param['comment']
                        nameContext = param['name']
                        prompt = param['prompt']
                        promptNeeded = 0 if param['promptNeeded'] == 'false' else 1
                        typeContext = param['type']
                        valueContext = param['value']
                        repositoryContextId = param['repositoryContextId']
                        
                        # Insert into aud_contextjob
                        insert_query = config.get_param('insert_queries', 'aud_contextjob')
                        params = (environementContextName, nameContext, prompt, promptNeeded, typeContext, valueContext, repositoryContextId, project_name, job_name, execution_date)
                        db.insert_data(insert_query, 'aud_contextjob', params)
                        print(f"Inserted data for context: {environementContextName}, param: {nameContext} into aud_contextjob")
                        
                        # Step 8: Insert context metadata into aud_contextgroupdetail
                        insert_query = config.get_param('insert_queries', 'aud_contextgroupdetail')
                        params = (nameContext, comment, project_name, environementContextName, execution_date)
                        db.insert_data(insert_query, 'aud_contextgroupdetail', params)
                        print(f"Inserted data for context: {environementContextName}, param: {nameContext} into aud_contextgroupdetail")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        if db:
            db.close()  # Ensure the database connection is closed

if __name__ == "__main__":
    main()


In [ ]:
import os
from config import Config  # Assuming Config class is defined in config.py
from database import Database  # Assuming Database class is defined in database.py
from XML_parse import XMLParser  # Importing the XMLParser class

def main():
    config_file = "configs/config.yaml"
    config = Config(config_file)
    items_directory = config.get_param('Directories', 'items_directory')

    db = None

    try:
        # Retrieve JDBC parameters and create a Database instance
        jdbc_params = config.get_jdbc_parameters()
        print("JDBC Parameters:", jdbc_params)  # Print JDBC parameters to debug

        db = Database(jdbc_params)
        db.set_jdbc_parameters(jdbc_params)  # Set JDBC parameters if needed
        db.connect_JDBC()  # Test the JDBC connection

        # Step 1: Get the execution date
        execution_date_query = config.get_param('queries', 'TRANSVERSE_QUERY_LASTEXECUTIONDATE')
        execution_date = db.get_execution_date(execution_date_query)
        print("Execution Date:", execution_date)

        # Step 2: Execute LOCAL_TO_DBBRUT_QUERY
        local_to_dbbrut_query = config.get_param('queries', 'LOCAL_TO_DBBRUT_QUERY')
        print("Executing query:", local_to_dbbrut_query)  # Print the query before execution
        local_to_dbbrut_query_results = db.execute_query(local_to_dbbrut_query)
        print("local_to_dbbrut_query_results:", local_to_dbbrut_query_results)

        # Step 3: Delete the output from aud_node based on the query results
        for result in local_to_dbbrut_query_results:
            project_name, job_name, _, _, _ = result  # Assuming result contains these fields in order
            
            # Prepare the conditions dictionary
            conditions = {
                'NameProject': project_name,
                'NameJob': job_name
            }
            
            db.delete_records('aud_node', **conditions)
            print(f"Deleted records for PROJECT_NAME from aud_node: {project_name}, JOB_NAME: {job_name}")

        # Step 4: Execute aud_node query
        aud_node_query = config.get_param('queries', 'aud_node')
        print("Executing query:", aud_node_query)  # Print the query before execution
        aud_node_results = db.execute_query(aud_node_query)
        print("aud_node_results:", aud_node_results)

        # Step 5: Delete the output from aud_contextjob based on the query results
        for result in aud_node_results:
            project_name, job_name = result
            conditions = {
                'NameProject': project_name,
                'NameJob': job_name
            }
            db.delete_records('aud_contextjob', **conditions)
            print(f"Deleted records for PROJECT_NAME from aud_contextjob: {project_name}, JOB_NAME: {job_name}")

        # Step 6: Loop over `.item` files in the items_directory and parse them
        for filename in os.listdir(items_directory):
            if filename.endswith('.item'):
                file_path = os.path.join(items_directory, filename)
                parts = filename.split('.', 1)  # Split at the first dot
                project_name = parts[0]
                job_name = parts[1].replace('.item', '') if len(parts) > 1 else None
                print("parts:", parts)
                print("project_name:", project_name)
                print("job_name:", job_name)
                print(f"Parsing file: {file_path}")
                xml_parser = XMLParser(file_path)
                parsed_data = xml_parser.parse_nodes()
                
                # Step 7: Insert parsed data into the `aud_element_node` table
                for data in parsed_data:
                    componentName = data['componentName']
                    componentVersion = data['componentVersion']
                    offsetLabelX = data['offsetLabelX']
                    offsetLabelY = data['offsetLabelY']
                    posX = data['posX']
                    posY = data['posY']
                    field = data['field']
                    name = data['name']
                    show = data['show']
                    value = data['value']
                    
                    # Adjust the value of `Componement_UniqueName` as needed, if it's derived from `componentName`
                    Componement_UniqueName = value if field == 'TEXT' and name == 'UNIQUE_NAME' else Componement_UniqueName
                    insert_query = config.get_param('insert_queries', 'aud_node')
                    params = (componentName, componentVersion, offsetLabelX, offsetLabelY, posX, posY, Componement_UniqueName, project_name, job_name, execution_date)
                    db.insert_data(insert_query, 'aud_node', params)
                    print(f"Inserted data for component: {data['componentName']} into aud_node")
                    
        # Step 8: Execute NodeJoinElementnode
        NodeJoinElementnode_query = config.get_param('queries', 'NodeJoinElementnode')
        print("Executing query:", NodeJoinElementnode_query)  # Print the query before execution
        NodeJoinElementnode_results = db.execute_query(NodeJoinElementnode_query)
        print("NodeJoinElementnode_results:", NodeJoinElementnode_results)
        
        # Step 9: Delete the output from aud_node based on the query results
        for result in NodeJoinElementnode_results:
            project_name, job_name, aud_componentValue = result  # Adjusted to three fields
            
            # Prepare the conditions dictionary
            conditions = {
                'NameProject': project_name,
                'NameJob': job_name,
                'aud_componentValue': aud_componentValue
            }
            
            db.delete_records('aud_node', **conditions)
            print(f"Deleted records for PROJECT_NAME from aud_node: {project_name}, JOB_NAME: {job_name}, COMPONENT_VALUE: {aud_componentValue}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        if db:
            db.close()  # Ensure the database connection is closed

if __name__ == "__main__":
    main()


In [ ]:
import os
from config import Config  # Assuming Config class is defined in config.py
from database import Database  # Assuming Database class is defined in database.py
from XML_parse import XMLParser  # Importing the XMLParser class

def main():
    config_file = "configs/config.yaml"
    config = Config(config_file)
    items_directory = config.get_param('Directories', 'items_directory')

    db = None

    try:
        # Retrieve JDBC parameters and create a Database instance
        jdbc_params = config.get_jdbc_parameters()
        print("JDBC Parameters:", jdbc_params)  # Print JDBC parameters to debug

        db = Database(jdbc_params)
        db.set_jdbc_parameters(jdbc_params)  # Set JDBC parameters if needed
        db.connect_JDBC()  # Test the JDBC connection

        # Step 1: Get the execution date
        execution_date_query = config.get_param('queries', 'TRANSVERSE_QUERY_LASTEXECUTIONDATE')
        execution_date = db.get_execution_date(execution_date_query)
        print("Execution Date:", execution_date)

        # Step 2: Execute LOCAL_TO_DBBRUT_QUERY
        local_to_dbbrut_query = config.get_param('queries', 'LOCAL_TO_DBBRUT_QUERY')
        print("Executing query:", local_to_dbbrut_query)  # Print the query before execution
        local_to_dbbrut_query_results = db.execute_query(local_to_dbbrut_query)
        print("local_to_dbbrut_query_results:", local_to_dbbrut_query_results)

        # Step 3: Delete the output from aud_bigdata based on the query results
        for result in local_to_dbbrut_query_results:
            project_name, job_name, _, _, _ = result  # Assuming result contains these fields in order
            
            # Prepare the conditions dictionary
            conditions = {
                'NameProject': project_name,
                'NameJob': job_name
            }
            
            db.delete_records('aud_bigdata', **conditions)
            print(f"Deleted records for PROJECT_NAME from aud_bigdata: {project_name}, JOB_NAME: {job_name}")

        # Step 4: Execute aud_bigdata query
        aud_bigdata_query = config.get_param('queries', 'aud_bigdata')
        print("Executing query:", aud_bigdata_query)  # Print the query before execution
        aud_bigdata_results = db.execute_query(aud_bigdata_query)
        print("aud_bigdata_results:", aud_bigdata_results)

        # Step 5: Delete the output from aud_contextjob based on the query results
        for result in aud_bigdata_results:
            project_name, job_name = result
            conditions = {
                'NameProject': project_name,
                'NameJob': job_name
            }
            db.delete_records('aud_contextjob', **conditions)
            print(f"Deleted records for PROJECT_NAME from aud_bigdata: {project_name}, JOB_NAME: {job_name}")

        # Step 6: Loop over `.item` files in the items_directory and parse them
        for filename in os.listdir(items_directory):
            if filename.endswith('.item'):
                file_path = os.path.join(items_directory, filename)
                parts = filename.split('.', 1)  # Split at the first dot
                project_name = parts[0]
                job_name = parts[1].replace('.item', '') if len(parts) > 1 else None
                print("parts:", parts)
                print("project_name:", project_name)
                print("job_name:", job_name)
                print(f"Parsing file: {file_path}")
                xml_parser = XMLParser(file_path)
                parameters_data = xml_parser.parse_parameters()
                
                # Step 7: Insert parsed parameters data into the `aud_bigdata` table
                for param_data in parameters_data:
                    field = param_data['field']
                    name = param_data['name']
                    show = param_data['show']
                    value = param_data['value']
                    
                    insert_query = config.get_param('insert_queries', 'aud_bigdata')
                    params = (field, name, show, value, project_name, job_name, execution_date)
                    db.insert_data(insert_query, 'aud_bigdata', params)
                    print(f"Inserted parameter data into aud_bigdata: {param_data}")

                    # Step 8: Insert element values into the `aud_bigdata_elementvalue` table
                    elementValues = param_data['elementValues']
                    for elementValue in elementValues:
                        elementRef = elementValue['elementRef']
                        elementRef_value = elementValue['value']
                        
                        insert_query = config.get_param('insert_queries', 'aud_bigdata_elementvalue')
                        params = (elementRef, elementRef_value, name, project_name, job_name, execution_date)
                        db.insert_data(insert_query, 'aud_bigdata_elementvalue', params)
                        print(f"Inserted element value data into aud_bigdata_elementvalue: {elementValue}")

    finally:
        if db:
            db.close()  # Ensure the database connection is closed

if __name__ == "__main__":
    main()


In [ ]:
import os
from config import Config  # Assuming Config class is defined in config.py
from database import Database  # Assuming Database class is defined in database.py
from XML_parse import XMLParser  # Importing the XMLParser class
import time

def main():
    config_file = "configs/config.yaml"
    config = Config(config_file)
    items_directory = config.get_param('Directories', 'items_directory')

    db = None

    try:
        # Retrieve JDBC parameters and create a Database instance
        jdbc_params = config.get_jdbc_parameters()
        print("JDBC Parameters:", jdbc_params)  # Print JDBC parameters to debug

        db = Database(jdbc_params)
        db.set_jdbc_parameters(jdbc_params)  # Set JDBC parameters if needed
        db.connect_JDBC()  # Test the JDBC connection

        # Step 1: Get the execution date
        execution_date_query = config.get_param('queries', 'TRANSVERSE_QUERY_LASTEXECUTIONDATE')
        execution_date = db.get_execution_date(execution_date_query)
        print("Execution Date:", execution_date)

        # Step 2: Execute LOCAL_TO_DBBRUT_QUERY
        local_to_dbbrut_query = config.get_param('queries', 'LOCAL_TO_DBBRUT_QUERY')
        print("Executing query:", local_to_dbbrut_query)  # Print the query before execution
        local_to_dbbrut_query_results = db.execute_query(local_to_dbbrut_query)
        print("local_to_dbbrut_query_results:", local_to_dbbrut_query_results)

        # Step 3: Delete the output from aud_metadata based on the query results
        for result in local_to_dbbrut_query_results:
            project_name, job_name, _, _, _ = result  # Assuming result contains these fields in order
            
            # Prepare the conditions dictionary
            conditions = {
                'NameProject': project_name,
                'NameJob': job_name
            }
            
            db.delete_records('aud_metadata', **conditions)
            print(f"Deleted records for PROJECT_NAME from aud_metadata: {project_name}, JOB_NAME: {job_name}")

        # Step 4: Execute aud_metadata query
        aud_metadata_query = config.get_param('queries', 'aud_metadata')
        print("Executing query:", aud_metadata_query)  # Print the query before execution
        aud_metadata_results = db.execute_query(aud_metadata_query)
        print("aud_metadata_results:", aud_metadata_results)

        # Step 5: Delete the output from aud_contextjob based on the query results
        for result in aud_metadata_results:
            project_name, job_name = result
            conditions = {
                'NameProject': project_name,
                'NameJob': job_name
            }
            db.delete_records('aud_contextjob', **conditions)
            print(f"Deleted records for PROJECT_NAME from aud_metadata: {project_name}, JOB_NAME: {job_name}")

        # Step 6: Loop over `.item` files in the items_directory and parse them
        filenames = [f for f in os.listdir(items_directory) if f.endswith('.item')]
        
        for filename in filenames:
            file_path = os.path.join(items_directory, filename)
            parts = filename.split('.', 1)  # Split at the first dot
            project_name = parts[0]
            job_name = parts[1].replace('.item', '') if len(parts) > 1 else None
            
            print("Processing file:", filename)
            print("parts:", parts)
            print("project_name:", project_name)
            print("job_name:", job_name)
            print(f"Parsing file: {file_path}")

            xml_parser = XMLParser(file_path)
            data = xml_parser._parse_file()
            print("Parsed Data:")
            print(data)

            # Step 7: Insert parsed parameters data into the `aud_metadata` table
            insert_query = config.get_param('insert_queries', 'aud_metadata')

            for node_data in data['nodes']:
                for elem_param in node_data['elementParameters']:
                    for elem_value in elem_param['elementValues']:
                        for meta in node_data['metadata']:
                            for column in meta['columns']:
                                params = (
                                    meta['connector'],
                                    meta['label'],
                                    meta['name'],
                                    column['comment'],
                                    0 if column['key'] == 'false' else 1,
                                    column['length'],
                                    column['name'],
                                    0 if column['nullable'] == 'false' else 1,
                                    column['pattern'],
                                    column['precision'],
                                    column['sourceType'],
                                    column['type'],
                                    0 if column['usefulColumn'] == 'false' else 1,
                                    column['originalLength'],
                                    column['defaultValue'],
                                    elem_value['value'],
                                    node_data['componentName'],
                                    project_name,
                                    job_name,
                                    execution_date
                                )
                                print(params)
                                db.insert_data(insert_query, 'aud_metadata', params)
                                time.sleep(0.001)
                                print(f"Inserted parameter data into aud_metadata: {params}")

        # Step 8: Execute MetadataJoinElementNode and delete results from aud_metadata
        metadata_join_query = config.get_param('queries', 'MetadataJoinElemntnode')
        print("Executing MetadataJoinElementNode query:", metadata_join_query)
        metadata_join_results = db.execute_query(metadata_join_query)
        print("MetadataJoinElementNode Results:", metadata_join_results)

        for result in metadata_join_results:
            conditions = {
            'NameProject': result[0],
            'NameJob': result[1],
            'aud_componentValue': result[2]
        }
            db.delete_records('aud_metadata', **conditions)
            print(f"Deleted records from aud_metadata based on MetadataJoinElementNode results: {conditions}")
    finally:
        if db:
            db.close()  # Ensure the database connection is closed

if __name__ == "__main__":
    main()


In [ ]:
import os
import time
from config import Config  # Assuming Config class is defined in config.py
from database import Database  # Assuming Database class is defined in database.py
from XML_parse import XMLParser  # Importing the XMLParser class


config_file = "configs/config.yaml"
config = Config(config_file)
items_directory = config.get_param('Directories', 'items_directory')
db = None
xml_parser = XMLParser()
parsed_files_data = xml_parser.loop_parse(items_directory)



def process_database_operations(config, parsed_files_data):
    """
    Perform various database operations including retrieving JDBC parameters, 
    executing queries, deleting records, and inserting data.

    Args:
        config (Config): An instance of the Config class for retrieving configuration parameters.
        parsed_files_data (list of tuples): A list where each tuple contains (project_name, job_name, parsed_data).
    """
    db = None

    try:
        # Retrieve JDBC parameters and create a Database instance
        jdbc_params = config.get_jdbc_parameters()
        print("JDBC Parameters:", jdbc_params)  # Print JDBC parameters to debug

        db = Database(jdbc_params)
        db.set_jdbc_parameters(jdbc_params)  # Set JDBC parameters if needed
        db.connect_JDBC()  # Test the JDBC connection

        # Step 1: Get the execution date
        execution_date_query = config.get_param('queries', 'TRANSVERSE_QUERY_LASTEXECUTIONDATE')
        execution_date = db.get_execution_date(execution_date_query)
        print("Execution Date:", execution_date)

        # Step 2: Execute LOCAL_TO_DBBRUT_QUERY
        local_to_dbbrut_query = config.get_param('queries', 'LOCAL_TO_DBBRUT_QUERY')
        print("Executing query:", local_to_dbbrut_query)  # Print the query before execution
        local_to_dbbrut_query_results = db.execute_query(local_to_dbbrut_query)
        print("local_to_dbbrut_query_results:", local_to_dbbrut_query_results)

        # Step 3: Delete the output from aud_metadata based on the query results
        for result in local_to_dbbrut_query_results:
            project_name, job_name, *_ = result  # Assuming result contains these fields in order
            conditions = {'NameProject': project_name, 'NameJob': job_name}
            db.delete_records('aud_metadata', **conditions)
            print(f"Deleted records for PROJECT_NAME from aud_metadata: {project_name}, JOB_NAME: {job_name}")

        # Step 4: Execute aud_metadata query
        aud_metadata_query = config.get_param('queries', 'aud_metadata')
        print("Executing query:", aud_metadata_query)  # Print the query before execution
        aud_metadata_results = db.execute_query(aud_metadata_query)
        print("aud_metadata_results:", aud_metadata_results)

        # Step 5: Delete the output from aud_contextjob based on the query results
        for result in aud_metadata_results:
            project_name, job_name = result
            conditions = {'NameProject': project_name, 'NameJob': job_name}
            db.delete_records('aud_contextjob', **conditions)
            print(f"Deleted records for PROJECT_NAME from aud_metadata: {project_name}, JOB_NAME: {job_name}")

        # Step 7: Insert parsed parameters data into the `aud_metadata` table
        insert_query = config.get_param('insert_queries', 'aud_metadata')
        for project_name, job_name, parsed_data in parsed_files_data:
            for node_data in parsed_data['nodes']:
                for elem_param in node_data['elementParameters']:
                    for elem_value in elem_param['elementValues']:
                        for meta in node_data['metadata']:
                            for column in meta['columns']:
                                params = (
                                    meta['connector'],
                                    meta['label'],
                                    meta['name'],
                                    column['comment'],
                                    0 if column['key'] == 'false' else 1,
                                    column['length'],
                                    column['name'],
                                    0 if column['nullable'] == 'false' else 1,
                                    column['pattern'],
                                    column['precision'],
                                    column['sourceType'],
                                    column['type'],
                                    0 if column['usefulColumn'] == 'false' else 1,
                                    column['originalLength'],
                                    column['defaultValue'],
                                    elem_value['value'],
                                    node_data['componentName'],
                                    project_name,
                                    job_name,
                                    execution_date
                                )
                                print(params)
                                db.insert_data(insert_query, 'aud_metadata', params)
                                time.sleep(0.001)
                                print(f"Inserted parameter data into aud_metadata: {params}")

        # Step 8: Execute MetadataJoinElementNode and delete results from aud_metadata
        metadata_join_query = config.get_param('queries', 'MetadataJoinElemntnode')
        print("Executing MetadataJoinElementNode query:", metadata_join_query)
        metadata_join_results = db.execute_query(metadata_join_query)
        print("MetadataJoinElementNode Results:", metadata_join_results)

        for result in metadata_join_results:
            conditions = {
                'NameProject': result[0],
                'NameJob': result[1],
                'aud_componentValue': result[2]
            }
            db.delete_records('aud_metadata', **conditions)
            print(f"Deleted records from aud_metadata based on MetadataJoinElementNode results: {conditions}")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if db:
            db.close()  # Ensure the database connection is closed






In [1]:
import sys
import os

# Add the directory containing AUD_304_ALIMMETADATA.py to the Python path
sys.path.append(os.path.dirname('C:/Users/sonia/Desktop:/SQOPS_SERVER_PY/SQOPS_SERVER_PY/Local_to_brut/AUD_302_ALIMCONTEXTJOB.py'))



In [3]:
from AUD_319_ALIMDOCCONTEXTGROUP import AUD_319_ALIMDOCCONTEXTGROUP
from config import Config  # Assuming Config class is defined in config.py
from XML_parse import XMLParser  # Importing the XMLParser class
from database import Database  # Assuming Database class is defined in database.py
import logging 

def main():
    config_file = "config.yaml"
    config = Config(config_file)

    
    # Retrieve JDBC parameters and create a Database instance
    jdbc_params = config.get_jdbc_parameters()
    print("JDBC Parameters:", jdbc_params)  # Print JDBC parameters to debug

    # Initialize Database instanc
    db = Database(jdbc_params)
    db.set_jdbc_parameters(jdbc_params)  # Set JDBC parameters if needed
    db.connect_JDBC()  # Test the JDBC connection


    # Step 1: Get the execution date
    execution_date_query = config.get_param('queries', 'TRANSVERSE_QUERY_LASTEXECUTIONDATE')
    execution_date = db.get_execution_date(execution_date_query)
    logging.info(f"Execution Date: {execution_date}")
    items_directory = config.get_param('Directories', 'items_directory')
    # Step 2: Execute LOCAL_TO_DBBRUT_QUERY
    local_to_dbbrut_query = config.get_param('queries', 'LOCAL_TO_DBBRUT_QUERY')
    logging.info(f"Executing query: {local_to_dbbrut_query}")
    local_to_dbbrut_query_results = db.execute_query(local_to_dbbrut_query)
    logging.debug(f"local_to_dbbrut_query_results: {local_to_dbbrut_query_results}")
    # Initialize XMLParser and parse files
    xml_parser = XMLParser()  # Initialize with reqauired arguments if needed
    parsed_files_data = xml_parser.loop_parse_screenshots(items_directory)
    print(parsed_files_data)

    
    # Call the function with the configuration, database instance, and parsed data
    # AUD_319_ALIMDOCCONTEXTGROUP(config, db, parsed_files_data)

if __name__ == "__main__":
    main()


Loaded configuration: {'Audit_JDBC': {'AUDIT_JDBC_connection_userPassword_password': 'Sa*201299', 'AUDIT_JDBC_drivers': 'C:/Users/sonia/Downloads/mysql-connector-j-9.0.0/mysql-connector-j-9.0.0/mysql-connector-j-9.0.0.jar', 'AUDIT_JDBC_connection_driverClass': 'com.mysql.cj.jdbc.Driver', 'AUDIT_JDBC_mappingFile': 'mysql_id', 'AUDIT_JDBC_connection_userPassword_userId': 'root', 'AUDIT_JDBC_connection_jdbcUrl': 'jdbc:mysql://localhost:3306/sqops_dataraise?allowLoadLocalInfile=true'}, 'queries': {'LOCAL_TO_DBBRUT_QUERY': 'select distinct PROJECT_NAME, JOB_NAME, JOB_PATH, JOB_VERSION, Talend_Version from audit_jobs_delta where talend_version is not null and niveau is not null', 'LOCAL_TO_DBBRUT_FILTER_name_elementNode': "where aud_nameElementNode in ('DBNAME','TYPE','QUERY','TABLE','FILENAME','TEMPDIR','sql','query')", 'LOCAL_TO_DBBRUT_FILTER_columnName_metadata': "where aud_columnName not in ('errorCode','errorMessage')", 'LOCAL_TO_DBBRUT_QUERY_ROUTINES': 'select distinct PROJECT_NAME, RO

In [1]:
import os
from config import Config  # Assuming Config class is defined in config.py
from XML_parse import XMLParser  # Importing the XMLParser class
from database import Database  # Assuming Database class is defined in database.py
import logging

def main():
    config_file = "config.yaml"
    config = Config(config_file)

    # Retrieve JDBC parameters and create a Database instance
    jdbc_params = config.get_jdbc_parameters()
    logging.info("JDBC Parameters: %s", jdbc_params)  # Log JDBC parameters for debugging

    # Step 3: Parse screenshot files from the directory
    items_directory = config.get_param('Directories', 'items_directory')

    if not os.path.isdir(items_directory):
        logging.error("Directory not found: %s", items_directory)
        return

    xml_parser = XMLParser()

    # Assuming the `loop_parse_screenshots` method parses all screenshot XMLs in the directory
    parsed_files_data = xml_parser.loop_parse_screenshots(items_directory)
    
    if parsed_files_data:
        logging.info("Parsed screenshot data: %s", parsed_files_data)  # Log parsed data
    else:
        logging.warning("No screenshot data found.")

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)  # Set up logging
    main()


Loaded configuration: {'Audit_JDBC': {'AUDIT_JDBC_connection_userPassword_password': 'Sa*201299', 'AUDIT_JDBC_drivers': 'C:/Users/sonia/Downloads/mysql-connector-j-9.0.0/mysql-connector-j-9.0.0/mysql-connector-j-9.0.0.jar', 'AUDIT_JDBC_connection_driverClass': 'com.mysql.cj.jdbc.Driver', 'AUDIT_JDBC_mappingFile': 'mysql_id', 'AUDIT_JDBC_connection_userPassword_userId': 'root', 'AUDIT_JDBC_connection_jdbcUrl': 'jdbc:mysql://localhost:3306/sqops_dataraise?allowLoadLocalInfile=true'}, 'queries': {'LOCAL_TO_DBBRUT_QUERY': 'select distinct PROJECT_NAME, JOB_NAME, JOB_PATH, JOB_VERSION, Talend_Version from audit_jobs_delta where talend_version is not null and niveau is not null', 'LOCAL_TO_DBBRUT_FILTER_name_elementNode': "where aud_nameElementNode in ('DBNAME','TYPE','QUERY','TABLE','FILENAME','TEMPDIR','sql','query')", 'LOCAL_TO_DBBRUT_FILTER_columnName_metadata': "where aud_columnName not in ('errorCode','errorMessage')", 'LOCAL_TO_DBBRUT_QUERY_ROUTINES': 'select distinct PROJECT_NAME, RO